In [29]:
import requests
import json
from collections import defaultdict
import pandas as pd

#中英對照
name_map = {
    "Temperature": "溫度",
    "DewPoint": "露點溫度",
    "RelativeHumidity": "相對濕度",
    "ApparentTemperature": "體感溫度",
    "ComfortIndex": "舒適度",
    "ComfortIndexDescription": "舒適度描述",
    "WindSpeed": "風速",
    "BeaufortScale": "蒲福風級",
    "ProbabilityOfPrecipitation": "降雨機率",
    "Weather": "天氣現象",
    "WeatherDescription": "天氣描述",
    "WindDirection": "風向",
    "WeatherCode": "天氣代碼"
}

In [30]:
def translate_name(be_translated_name):
    ch_name = name_map.get(be_translated_name, be_translated_name)
    return ch_name

url = f"https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-D0047-069"
api_token = "CWA-8917E0F0-A194-4359-9634-D8AF9CDBB872"
def get_weather(city_name, district_name, show_top_ndata=10):
    params = {
        "Authorization" : api_token,
        "format": "JSON",
        "LocationName" : district_name
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        print("臺灣各縣市鄉鎮未來3天天氣預報")
        cities = data['records']['Locations']
        found_city = False
        found_district = False

        for city in cities:
            if city['LocationsName'] == city_name:
                found_city = True
                for district in city['Location']:
                    if district['LocationName'] == district_name:
                        found_district = True
                        print(f"\n📍 {city_name} {district_name}")

                        weather_elements = district['WeatherElement']
                        weather_merged = defaultdict(dict)

                        for elem in weather_elements:
                            elem_name = elem['ElementName']
                            for time_list in elem['Time'][:show_top_ndata]:
                                # 自動偵測是 DataTime 或 StartTime
                                time = time_list.get('DataTime') or time_list.get('StartTime')
                                element_value = time_list['ElementValue']
                                if element_value:
                                    for k, v in element_value[0].items():#.items()回傳(key, value)列表
                                        weather_merged[time][k] = v

                        # 建立表格資料
                        desired_order = [
                            "時間", "溫度", "露點溫度", "相對濕度", "體感溫度", "舒適度", "舒適度描述",
                            "風速", "蒲福風級", "風向", "降雨機率", "天氣現象", "天氣代碼", "天氣描述"
                        ]
                        rows = []
                        for time in sorted(weather_merged.keys())[:show_top_ndata]:
                            row = {"時間": time}
                            for k, v in weather_merged[time].items():
                                row[translate_name(k)] = v
                            rows.append(row)

                        # 轉為 DataFrame
                        # 轉為 DataFrame
                        df = pd.DataFrame(rows)

                        # 👉 補上缺欄，重新排序
                        for col in desired_order:
                            if col not in df.columns:
                                df[col] = pd.NA
                        df = df[desired_order]
                        df = df.fillna("未測量")
                        return df  #jupyter表格
                        '''
                        # 顯示表格
                        print(df.to_string(index=False))   #純文字輸出   
                        return   
                        '''
                        
                        
                                                                                                                                                   
        print(f"⚠️ 找不到 {district_name} 的天氣資料。")

    except requests.exceptions.HTTPError as http_err:
        print("❌ HTTP 錯誤：", http_err)
    except requests.exceptions.RequestException as req_err:
        print("❌ 其他請求錯誤（例如網路錯誤）", req_err)
    except KeyError as ke:
        print("❌ 回傳資料格式錯誤，缺少欄位：", ke)
    except Exception as e:
        print("❌ 其他非 requests 錯誤：", e)  

In [31]:
'''
if __name__ == "__main__":
    
    get_weather("新北市", "深坑區", show_top_ndata=10)  #純文字
'''
df = get_weather("新北市", "深坑區", show_top_ndata=12)  #jupyter表格
df  # 直接放在 cell 最後，Jupyter 會自動以表格顯示

臺灣各縣市鄉鎮未來3天天氣預報

📍 新北市 深坑區


,時間,溫度,露點溫度,相對濕度,體感溫度,舒適度,舒適度描述,風速,蒲福風級,風向,降雨機率,天氣現象,天氣代碼,天氣描述
0,2025-06-19T12:00:00+08:00,33,29,79,39,31,易中暑,2,2,東南風,30,午後短暫雷陣雨,15,午後短暫雷陣雨。降雨機率30%。溫度攝氏32至33度。易中暑。東南風 平均風速1-2級(每秒...
1,2025-06-19T13:00:00+08:00,33,29,83,40,31,易中暑,未測量,未測量,未測量,未測量,未測量,未測量,未測量
2,2025-06-19T14:00:00+08:00,32,29,85,39,31,易中暑,未測量,未測量,未測量,未測量,未測量,未測量,未測量
3,2025-06-19T15:00:00+08:00,31,29,86,37,30,悶熱,2,2,西南風,30,午後短暫雷陣雨,15,午後短暫雷陣雨。降雨機率30%。溫度攝氏30至31度。悶熱。西南風 平均風速1-2級(每秒2...
4,2025-06-19T16:00:00+08:00,30,27,85,36,29,悶熱,未測量,未測量,未測量,未測量,未測量,未測量,未測量
5,2025-06-19T17:00:00+08:00,30,27,85,35,28,悶熱,未測量,未測量,未測量,未測量,未測量,未測量,未測量
6,2025-06-19T18:00:00+08:00,29,26,86,34,28,悶熱,2,2,偏西風,20,多雲,04,多雲。降雨機率20%。溫度攝氏27至29度。悶熱。偏西風 平均風速1-2級(每秒2公尺)。相...
7,2025-06-19T19:00:00+08:00,28,26,88,34,27,悶熱,未測量,未測量,未測量,未測量,未測量,未測量,未測量
8,2025-06-19T20:00:00+08:00,27,26,90,33,27,悶熱,未測量,未測量,未測量,未測量,未測量,未測量,未測量
9,2025-06-19T21:00:00+08:00,27,25,91,32,26,舒適,1,<= 1,東南風,20,多雲,04,多雲。降雨機率20%。溫度攝氏26至27度。舒適。東南風 平均風速<= 1級(每秒1公尺)。...
